In [ ]:
import netCDF4 
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import calendar
import matplotlib.pyplot as plt

In [ ]:
# 创建一个空数组，用于存储每个月份的降水数据
monthly_precipitation = []

# 遍历从1到12的月份
for month in range(1, 13):
    # 创建一个空数组，用于存储每年该月份的降水数据
    monthly_data = []

        # 构造文件路径
    filepath = f"/Users/ynbsztl/dataset/precipitation/tmp_2021.nc"
    
        # 读取NetCDF文件
    nc = netCDF4.Dataset(filepath)
    
    # 提取气温
    precipitation = nc.variables['tmp'][month - 1, :, :]
    
    # 添加到月份数据数组
    monthly_data.append(precipitation)
    
    # 计算月份的降水均值并添加到数组中
    monthly_mean = np.mean(monthly_data, axis=0)
    monthly_precipitation.append(monthly_mean)

# 绘制图形
fig, axs = plt.subplots(4, 3, figsize=(12, 12))

# 循环绘制每个月的图像
for month, ax in enumerate(axs.flat):
    # 获取对应月份的降水均值数据
    precipitation_mean = monthly_precipitation[month]

    # 将图像数据逆时针旋转90度
    rotated_image = np.rot90(precipitation_mean, k=-1)
        
        # 水平翻转图像
    flipped_image = cv2.flip(rotated_image, 1)    
    # 绘制图像
    ax.imshow(flipped_image,cmap='viridis')
    ax.set_title(f"Month {month+1}")
    ax.axis('off')

# 调整子图之间的间距
plt.tight_layout()
plt.show()

In [ ]:
# 创建一个空数组，用于存储15个月的偏离均值程度数据
deviation_images = []

# 创建一个空数组，用于存储15个月的偏离均值程度数据
deviation_images = []

# 遍历从2015年4月到2016年6月的日期范围
for year in range(2015, 2017):
    for month in range(1, 13):
        # 判断是否在范围内
        if (year == 2015 and month >= 4) or (year == 2016 and month <= 6):
            # 构造文件路径
            filepath = f"/Users/ynbsztl/dataset/precipitation/CN_Prec_MonthlyMean_1km_{year}.nc"

            # 读取NetCDF文件
            nc = netCDF4.Dataset(filepath)
            
            # 提取降水数据
            precipitation = nc.variables['pre'][month - 1, :, :]

            # 计算偏离均值程度
            deviation = (precipitation - monthly_precipitation[month - 1]) / np.std(monthly_precipitation[month - 1])

            # 添加到偏离均值程度数据数组
            deviation_images.append(deviation)


# 绘制图像
fig, axs = plt.subplots(5, 3, figsize=(12, 12))

# 循环绘制每个月的图像
for i, ax in enumerate(axs.flat):
    # 获取对应月份的偏离均值程度数据
    deviation_image = deviation_images[i]

    # 将图像数据逆时针旋转90度
    rotated_image = np.rot90(deviation_image, k=-1)

    # 水平翻转图像
    flipped_image = cv2.flip(rotated_image, 1)

    # 获取当前图像的年份和月份，2015.4 - 2016.6
    year = 2015 if i < 8 else 2016
    month = (i + 3) % 12 + 1
    month_name = calendar.month_abbr[month]
    title = f"{month_name}, {year}"

    # 绘制图像
    img = ax.imshow(flipped_image, cmap='viridis')
    ax.set_title(title)
    ax.axis('off')

    # 添加colorbar
    cbar = plt.colorbar(img, ax=ax,fraction=0.05)
    cbar.set_label('Deviation')

# 调整子图之间的间距
plt.tight_layout()
plt.show()

In [ ]:
# 统计大于或小于1个标准差的次数，如果是想让（X- mean）/Std.Dev = 1 应该是修改这里
counts = np.sum(np.abs(deviation_images) >= 1, axis=0)

# 将图像数据逆时针旋转90度
rotated_counts = np.rot90(counts, k=-1)

# 水平翻转图像
flipped_counts = cv2.flip(rotated_counts, 1)

# 绘制次数图像
fig, ax = plt.subplots(figsize=(10, 8))
img = ax.imshow(flipped_counts, cmap='viridis')

# 添加colorbar
cbar = plt.colorbar(img)
cbar.set_label('Counts')

# 设置标题和轴标签
ax.set_title('Occurrences of Deviation > 1 Standard Deviation')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

In [ ]:
deviation_images = np.array(deviation_images)

# 统计大于一个标准差的次数
counts_positive = np.sum(deviation_images >= 1, axis=0)

# 统计小于一个标准差的次数
counts_negative = np.sum(deviation_images <= -1, axis=0)

# 将图像数据逆时针旋转90度
rotated_counts_positive = np.rot90(counts_positive, k=-1)
rotated_counts_negative = np.rot90(counts_negative, k=-1)

# 水平翻转图像
flipped_counts_positive = cv2.flip(rotated_counts_positive, 1)
flipped_counts_negative = cv2.flip(rotated_counts_negative, 1)

# 创建一个包含两个子图的图形布局
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# 绘制正数次数图像
axs[0].imshow(flipped_counts_positive, cmap='viridis')
axs[0].set_title('Occurrences of Deviation > 1 Standard Deviation (Positive)')
axs[0].set_xlabel('Longitude')
axs[0].set_ylabel('Latitude')
cbar_positive = plt.colorbar(axs[0].images[0], ax=axs[0], fraction=0.025)
cbar_positive.set_label('Counts')

# 绘制负数次数图像
axs[1].imshow(flipped_counts_negative, cmap='viridis')
axs[1].set_title('Occurrences of Deviation < -1 Standard Deviation (Negative)')
axs[1].set_xlabel('Longitude')
axs[1].set_ylabel('Latitude')
cbar_negative = plt.colorbar(axs[1].images[0], ax=axs[1], fraction=0.025)
cbar_negative.set_label('Counts')

# 调整子图之间的间距
plt.tight_layout()

# 显示图形
plt.show()

In [ ]:
# 绘制偏差的大图
fig, ax = plt.subplots(figsize=(10, 8),dpi=3000)
img = ax.imshow(flipped_counts_positive, cmap='viridis')

# 添加colorbar
cbar = plt.colorbar(img)
cbar.set_label('Counts')

# 设置标题和轴标签
ax.set_title('Occurrences of Deviation > 1 Standard Deviation')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

In [ ]:
# 绘制偏差的大图
fig, ax = plt.subplots(figsize=(10, 8),dpi=300)
img = ax.imshow(flipped_counts_negative, cmap='viridis')

# 添加colorbar
cbar = plt.colorbar(img)
cbar.set_label('Counts')

# 设置标题和轴标签
ax.set_title('Occurrences of Deviation > -1 Standard Deviation')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()

In [ ]:
# 将二维数组展平为一维数组
counts_positive_flat = flipped_counts_positive.flatten()

# 绘制直方图
plt.hist(counts_positive_flat, bins=14, edgecolor='black')

# 设置标题和轴标签
plt.title('Histogram of Flipped Counts (Positive)')
plt.xlabel('Counts')
plt.ylabel('Frequency')

# 显示图形
plt.show()

In [ ]:
flipped_counts_positive_1 = np.where(flipped_counts_positive == 12, flipped_counts_positive, 0)

# 绘制偏差的大图
fig, ax = plt.subplots(figsize=(10, 8), dpi=3000)
img = ax.imshow(flipped_counts_positive_1, cmap='viridis')


# 设置标题和轴标签
ax.set_title('12 Month Deviated From Standard')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()